# 2: Observing the same object with multiple telescopes

A brief introduction into using ScopeSim to observe a cluster in the LMC using the 39m ELT and the 1.5m LFOA

First set up all relevant imports:

In [ ]:
import matplotlib.pyplot as plt

from scopesim import Simulation
from scopesim_templates.stellar.clusters import cluster

## Create a star cluster ``Source`` object

Now, create a star cluster using the scopesim_templates package. You can ignore the output that is sometimes printed. The seed argument is used to control the random number generation that creates the stars in the cluster. If this number is kept the same, the output will be consistent with each run, otherwise the position and brightness of the stars is randomised every time.

In [ ]:
source = cluster(
    mass=10000,       # Msun
    distance=50000,   # parsec
    core_radius=2.1,  # parsec
    seed=9001,        # random number seed
)

## Observe the source

### Observe with the [1.5m telescope at the LFOA](https://foa.univie.ac.at/instrumentation/)

<img src=https://foa.univie.ac.at/fileadmin/_processed_/csm_FOA-2_1e249577a3.jpg width=400, align=left>


In [ ]:
lfoa = Simulation("LFOA")
data_lfoa = lfoa(source, dit=10, ndit=360)[1].data

### Observe the same ``Source`` with MICADO at the ELT

<img src=https://astro.univie.ac.at/fileadmin/user_upload/i_astro/Aktuelles-News/2021/The_E-ELT.jpeg width=400 align=left>

In [ ]:
micado = Simulation("MICADO")
# Use the central detector of the full MICADO array
micado.optical_train["detector_window"].include = False
micado.optical_train["full_detector_array"].include = True
micado.optical_train["full_detector_array"].meta["active_detectors"] = [5]

data_micado = micado(source, dit=10, ndit=360)[1].data

## Plot up the results

LFOA has a larger field of view and a lower resolution than MICADO, so while LFOA shows the whole cluster, MICADO only shows the center.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

img_lfoa = ax1.imshow(data_lfoa, norm="log", origin="lower")
fig.colorbar(img_lfoa, ax=ax1)
ax1.set_title("1.5m LFOA")

img_micado = ax2.imshow(data_micado, norm="log", vmin=1e5, vmax=1e6, origin="lower")
fig.colorbar(img_micado, ax=ax2)
ax2.set_title("39m ELT")

### Zoom in the LFOA observation

We can use the pixel scale of LFOA and MICADO to cut out the exact piece of the LFOA observation that corresponds to the MICADO one.

In [ ]:
pixel_scale_lfoa = lfoa.settings["!INST.pixel_scale"]
pixel_scale_micado = micado.settings["!INST.pixel_scale"]
scale_factor = pixel_scale_lfoa / pixel_scale_micado

size_lfoa_x = data_micado.shape[0] / scale_factor /2
size_lfoa_y = data_micado.shape[1] / scale_factor /2

xcen_lfoa = data_lfoa.shape[0] / 2
ycen_lfoa = data_lfoa.shape[1] / 2

x_low = round(xcen_lfoa - size_lfoa_x / 2)
x_high = round(xcen_lfoa + size_lfoa_x / 2)
y_low = round(ycen_lfoa - size_lfoa_y / 2)
y_high = round(ycen_lfoa + size_lfoa_y / 2)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

img_lfoa = ax1.imshow(data_lfoa[x_low:x_high, y_low:y_high], norm="log", origin="lower")
fig.colorbar(img_lfoa, ax=ax1)
ax1.set_title("1.5m LFOA")

img_micado = ax2.imshow(data_micado, norm="log", vmin=1e5, vmax=1e6, origin="lower")
fig.colorbar(img_micado, ax=ax2)
ax2.set_title("39m ELT")

# fig.savefig("withfullarray.png")